# Modelo FruitScan - Comparación de ResNet34, ResNet36 y VGG19

Este notebook entrena y compara tres modelos de **Redes Neuronales Convolucionales (CNN)** utilizando un dataset comprimido en formato ZIP. El objetivo principal es evaluar el rendimiento de **ResNet18**, **ResNet34**, y **VGG19** en la clasificación de frutas frescas y podridas.

## Contenido:
- **Dataset:** Imágenes de frutas frescas y podridas (manzanas, bananas, naranjas).
- **Modelos:** ResNet18, ResNet34 y VGG19 (pre-entrenados y fine-tuned).
- **Evaluación:** Curvas de pérdida, precisión por época y matrices de confusión para cada modelo.

---

## Paso 1: Descarga y Extracción del Dataset

En este paso, se descarga el dataset de frutas directamente desde Google Drive y se extrae su contenido en el directorio `data/`. Esto asegura que las imágenes estén disponibles para el entrenamiento y la validación de los modelos.


In [ ]:
!pip install gdown

import gdown
import zipfile
import os

# Descargar directamente el ZIP desde Google Drive (sin necesidad de montar cuenta)
file_id = "1WEHp4vAUuu1wswR6tQSkxgC2shIpFIJS"
output = "fruits_dataset.zip"
gdown.download(id=file_id, output=output, quiet=False)

# Extraer contenido
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("data/")

# Confirmar clases
extract_path = "data/dataset/dataset/train"
print("✅ Clases detectadas:", os.listdir(extract_path))

---

## Paso 2: Importación de Librerías

Se importan todas las librerías necesarias para la manipulación de datos, la construcción y entrenamiento de modelos de PyTorch, y la visualización de resultados. Esto incluye `torch`, `torchvision`, `matplotlib`, y `sklearn`.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score
import numpy as np
import time

---

## Paso 3: Configuraciones Iniciales y Preparación del Dataset

Aquí se definen los parámetros generales como el tamaño del batch, las dimensiones de las imágenes y el número de épocas. Además, se configuran las **transformaciones de datos** necesarias (redimensionamiento, conversión a tensor y normalización) y se cargan los datasets de entrenamiento y prueba utilizando `ImageFolder` de `torchvision`.

In [ ]:
# Parámetros generales
batch_size = 8
img_size = 224
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = "data/dataset/dataset"

# Transformaciones
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset usando carpetas train/ y test/ directamente
train_dir = "data/dataset/dataset/train"
test_dir = "data/dataset/dataset/test"

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print("Clases detectadas:", class_names)

---

## Paso 4: Función de Entrenamiento Genérica

Se define una función `entrenar_modelo` que encapsula el bucle de entrenamiento y validación. Esta función es genérica y puede ser utilizada con cualquiera de los modelos de CNN. Calcula y retorna las pérdidas de entrenamiento y validación, las precisiones de validación, y las predicciones reales para la generación de la matriz de confusión.

In [ ]:
def entrenar_modelo(model, train_loader, val_loader, optimizer, criterion, epochs, nombre):
    model = model.to(device)
    train_losses, val_losses, val_accuracies = [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # Validación
        model.eval()
        val_loss = 0.0
        y_true, y_pred = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())

        val_loss /= len(val_loader)
        accuracy = accuracy_score(y_true, y_pred)
        val_losses.append(val_loss)
        val_accuracies.append(accuracy)

        print(f"\u2705 Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Accuracy: {accuracy:.2%}")

    return model, train_losses, val_losses, val_accuracies, y_true, y_pred

---

## Paso 5: Entrenamiento de Modelos

Aquí se instancian, modifican (para adaptar la capa de salida al número de clases) y entrenan los tres modelos pre-entrenados: **ResNet18**, **ResNet34**, y **VGG19**. Para cada modelo, se congela la mayoría de las capas pre-entrenadas y solo se entrena la capa de clasificación final. Se utiliza el optimizador Adam y la función de pérdida CrossEntropyLoss.

In [ ]:
# ResNet18
print("\n--- Entrenando ResNet18 ---")
resnet18 = models.resnet18(pretrained=True)
for param in resnet18.parameters():
    param.requires_grad = False
resnet18.fc = nn.Linear(resnet18.fc.in_features, len(class_names))

optimizer_r18 = optim.Adam(resnet18.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

resnet18, r18_train, r18_val, r18_acc, y_true_r18, y_pred_r18 = entrenar_modelo(
    resnet18, train_loader, val_loader, optimizer_r18, criterion, epochs, "ResNet18"
)

# ResNet34
print("\n--- Entrenando ResNet34 ---")
resnet34 = models.resnet34(pretrained=True)
for param in resnet34.parameters():
    param.requires_grad = False
resnet34.fc = nn.Linear(resnet34.fc.in_features, len(class_names))

optimizer_r34 = optim.Adam(resnet34.fc.parameters(), lr=0.001)

resnet34, r34_train, r34_val, r34_acc, y_true_r34, y_pred_r34 = entrenar_modelo(
    resnet34, train_loader, val_loader, optimizer_r34, criterion, epochs, "ResNet34"
)

# VGG19
print("\n--- Entrenando VGG19 ---")
vgg19 = models.vgg19(pretrained=True)
for param in vgg19.features.parameters():
    param.requires_grad = False
vgg19.classifier[6] = nn.Linear(vgg19.classifier[6].in_features, len(class_names))

optimizer_vgg = optim.Adam(vgg19.classifier.parameters(), lr=0.001)

vgg19, vgg_train, vgg_val, vgg_acc, y_true_vgg, y_pred_vgg = entrenar_modelo(
    vgg19, train_loader, val_loader, optimizer_vgg, criterion, epochs, "VGG19"
)

---

## Paso 6: Comparación de Métricas de los Modelos

Se generan gráficos para comparar visualmente las curvas de pérdida de validación y la precisión de validación a lo largo de las épocas para cada uno de los modelos. Además, se imprime la precisión final (macro average) de cada modelo para una comparación directa.

In [ ]:
def plot_metric_comparison(metric_dict, title, ylabel):
    plt.figure(figsize=(10, 5))
    for label, values in metric_dict.items():
        plt.plot(values, label=label)
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.show()

# Diccionarios con los valores por modelo
loss_val_dict = {
    "ResNet18": r18_val,
    "ResNet34": r34_val,
    "VGG19": vgg_val
}

acc_val_dict = {
    "ResNet18": r18_acc,
    "ResNet34": r34_acc,
    "VGG19": vgg_acc
}

# Cálculo de precisión por época no disponible directamente -> omitimos para no inventar valores
# Se podría hacer con batches pero complica la función, por ahora mostramos precisión final por modelo
def calcular_precision_final(y_true, y_pred):
    return precision_score(y_true, y_pred, average="macro")

precision_dict = {
    "ResNet18": calcular_precision_final(y_true_r18, y_pred_r18),
    "ResNet34": calcular_precision_final(y_true_r34, y_pred_r34),
    "VGG19": calcular_precision_final(y_true_vgg, y_pred_vgg),
}

plot_metric_comparison(loss_val_dict, "Validation Loss", "Loss")
plot_metric_comparison(acc_val_dict, "Validation Accuracy", "Accuracy")

# Mostrar precisión final
print("📌 Precision por modelo (macro avg):")
for modelo, prec in precision_dict.items():
    print(f"{modelo}: {prec:.4f}")

---

## Paso 6.1: Análisis Profundo y Determinación del Mejor Modelo

La fase de análisis es donde interpretamos los resultados numéricos y visuales para extraer conclusiones significativas sobre el rendimiento de cada modelo. Basándonos en los gráficos de curvas de pérdida de validación, la precisión de validación a lo largo de las épocas, y la precisión final (`macro avg`), podemos identificar el modelo que exhibe el rendimiento más robusto y generalizado para nuestro problema de clasificación de frutas frescas vs. podridas.

Para realizar esta determinación, nos enfocamos en las siguientes métricas clave:

1.  **Pérdida de Validación (`Validation Loss`)**: Este valor es un indicador crucial de cuán bien está aprendiendo el modelo y generalizando a datos no vistos. Un `Validation Loss` bajo y estable a lo largo de las épocas sugiere que el modelo no solo está aprendiendo de manera efectiva, sino que también evita el sobreajuste (`overfitting`). Una pérdida que disminuye continuamente en el entrenamiento pero aumenta en la validación es una señal clara de sobreajuste.

2.  **Precisión de Validación (`Validation Accuracy`)**: Esta métrica nos dice la proporción de predicciones correctas que el modelo realiza en el conjunto de validación. Un valor más alto indica una mejor capacidad de clasificación. Es importante observar no solo el valor final, sino también la trayectoria: ¿la precisión se estabiliza, sigue mejorando o comienza a disminuir (posible sobreajuste)?

3.  **Precisión Final (`Precision (macro avg)`)**: Como se explicó anteriormente, la precisión macro promedio es fundamental porque calcula la precisión para cada clase de forma independiente y luego promedia esos valores. Esto es especialmente valioso en datasets donde las clases pueden no estar balanceadas, ya que asegura que el rendimiento en clases minoritarias no sea ignorado por un buen rendimiento en clases mayoritarias. Una alta precisión macro promedio indica que el modelo es consistentemente bueno en todas las categorías de frutas.

### **Observaciones Típicas y Fundamentos Arquitectónicos:**

* **VGG19**: Esta arquitectura es conocida por su simplicidad (bloques de capas convolucionales y de *pooling* seguidos por capas densas) y su gran profundidad. Sin embargo, su enorme número de parámetros puede hacer que sea más lenta de entrenar y, en tareas de *transfer learning* donde solo se entrena la capa final, a veces puede no alcanzar la misma eficiencia o capacidad de adaptación que las arquitecturas con **conexiones residuales**. Su rendimiento puede ser bueno, pero a menudo se ve superado por las ResNets en datasets más complejos o con menos datos disponibles para el *fine-tuning* de la capa final.

* **ResNet18**: Como una de las arquitecturas ResNet más ligeras, ResNet18 introduce las innovadoras **conexiones residuales (skip connections)**. Estas conexiones permiten que la información pase directamente de capas anteriores a capas posteriores, mitigando el problema del *vanishing gradient* (desaparición del gradiente) en redes muy profundas. Esto facilita el entrenamiento y permite a la red aprender características más complejas. ResNet18 generalmente ofrece un excelente equilibrio entre un rendimiento competitivo y una eficiencia computacional razonable, lo que la convierte en una opción muy popular para *transfer learning*.

* **ResNet34**: Con más capas que ResNet18 (pero aún dentro de la misma familia de arquitecturas residuales), ResNet34 tiene una mayor capacidad de modelado. Esto le permite capturar patrones y características más intrincadas en las imágenes. En muchos casos, ResNet34 puede lograr una **precisión ligeramente superior** a ResNet18 y una **menor pérdida de validación**, aunque a costa de un tiempo de entrenamiento un poco mayor y un consumo de recursos computacionales marginalmente más alto. Su diseño intrínseco con bloques residuales le permite escalar bien la profundidad sin los problemas de optimización que enfrentan otras redes profundas como VGG sin estas conexiones.

**Conclusión y Razón del Mejor Rendimiento (basada en resultados esperados y experiencia general):**

Tras una revisión de los gráficos de *Validation Loss* y *Validation Accuracy*, así como la métrica numérica de *Precision (macro avg)*, se espera que el modelo **ResNet34** demuestre ser el de **mejor rendimiento** para esta tarea de clasificación de frutas.

Las razones principales de su superioridad probable son:
1.  **Profundidad Optimizada**: ResNet34 es lo suficientemente profunda como para aprender representaciones ricas y complejas de las imágenes, pero no tan profunda como para volverse intratable con el *transfer learning* en este dataset. Su arquitectura, con más bloques residuales que ResNet18, le confiere una mayor capacidad de aprendizaje.
2.  **Eficacia de las Conexiones Residuales**: Las *skip connections* de la familia ResNet son clave. Permiten que los gradientes fluyan más fácilmente a través de la red, lo que evita la saturación de gradientes y facilita el entrenamiento de capas más profundas. Esto es crucial para aprender las sutiles diferencias entre frutas frescas y podridas, incluso con cambios en texturas, colores o formas.
3.  **Generalización Mejorada**: Las ResNets, gracias a su diseño, tienden a generalizar mejor a datos no vistos, lo que se refleja en una menor pérdida de validación y una mayor precisión de validación en comparación con VGG19, que podría ser más propensa al sobreajuste o a una convergencia más lenta en este tipo de escenarios de *transfer learning*.

En resumen, el **ResNet34** logra un equilibrio óptimo entre complejidad de modelo y la capacidad de aprender características discriminativas para esta tarea, superando típicamente a sus contrapartes en términos de precisión y robustez. 

---

## Paso 7: Matrices de Confusión

Se generan y muestran las matrices de confusión normalizadas para cada modelo. Estas matrices permiten visualizar el rendimiento de clasificación por clase, identificando dónde los modelos aciertan y dónde se equivocan.

In [ ]:
def mostrar_confusion(y_true, y_pred, modelo):
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(8, 6))
    disp.plot(cmap='Blues', ax=ax, xticks_rotation=45)
    plt.title(f"Normalized Confusion Matrix - {modelo}")
    plt.show()

In [ ]:
mostrar_confusion(y_true_r18, y_pred_r18, "ResNet18")
mostrar_confusion(y_true_r34, y_pred_r34, "ResNet34")
mostrar_confusion(y_true_vgg, y_pred_vgg, "VGG19")

---

## Paso 8: Guardado de Modelos

Los pesos entrenados de cada modelo se guardan en archivos `.pth` separados. Esto permite recargar y reutilizar los modelos en el futuro sin necesidad de reentrenarlos.

In [ ]:
torch.save(resnet18.state_dict(), "model_resnet18.pth")
torch.save(resnet34.state_dict(), "model_resnet34.pth")
torch.save(vgg19.state_dict(), "model_vgg19.pth")

print("✅ Modelos guardados correctamente.")

---

## Paso 9: Mostrar Información sobre la Clase Predicha (Simulación)

Este paso simula cómo se podría usar la predicción de un modelo en una aplicación real, como una interfaz de usuario. Se proporciona información descriptiva para cada clase de fruta, demostrando cómo se podría integrar la salida del modelo con datos adicionales para el usuario final.

In [ ]:
fruit_info = {
    'freshapples': 'These are fresh apples. A good source of fiber and vitamin C.',
    'freshbanana': 'These are fresh bananas. High in potassium.',
    'freshoranges': 'These are fresh oranges. Rich in vitamin C and antioxidants.',
    'rottenapples': 'These are rotten apples. Not suitable for consumption.',
    'rottenbanana': 'These are rotten bananas. Sometimes usable for cooking, but handle with care.',
    'rottenoranges': 'These are rotten oranges. Should be discarded.'
}

# Simulación de uso posterior (por ejemplo, en Streamlit)
# Podrías reemplazar 'predicted_class_name' por la clase detectada en tiempo real
predicted_class_name = "freshapples"

if predicted_class_name in fruit_info:
    print(f"\nℹ️ Info for predicted fruit ({predicted_class_name}):")
    print(fruit_info[predicted_class_name])
else:
    print("\nℹ️ No info available for the predicted class.")